In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("output/smartmarket_leads_scope.csv", parse_dates=["date"])
camp = pd.read_csv("output/smartmarket_campaigns_kpi.csv")

In [2]:
quant_cols = ["cost","impressions","clicks","conversions","ctr","conv_rate","cpc","cpa","cost_per_lead","n_leads"]
uni_quant = camp[quant_cols].describe().T
uni_quant["median"] = camp[quant_cols].median()
uni_quant = uni_quant[["count","mean","median","std","min","max"]]
uni_quant

,count,mean,median,std,min,max
cost,12.0,3895.083333,3458.000000,2477.871794,1113.000000,9211.000000
impressions,12.0,97250.000000,101845.500000,35212.589918,44939.000000,149599.000000
clicks,12.0,2363.000000,2246.500000,1217.071821,753.000000,5023.000000
conversions,12.0,164.083333,160.000000,93.142567,8.000000,352.000000
ctr,12.0,0.023749,0.022925,0.005484,0.015730,0.033576
conv_rate,12.0,0.065948,0.066443,0.020371,0.010624,0.091772
cpc,12.0,1.803104,1.602838,1.095526,0.717497,4.132646
cpa,12.0,37.021614,22.504926,44.208620,10.443182,170.000000
cost_per_lead,12.0,3.508650,2.796888,2.905414,0.930869,9.990239
n_leads,12.0,1250.000000,1308.500000,226.139539,922.000000,1461.000000


In [3]:
uni_quant.to_csv("02_results/univarie_quant_campaigns.csv", index=True)

In [4]:
qual_cols = ["channel","device","company_size","sector","region","status"]

freq_tables = {}
for c in qual_cols:
    t = df[c].value_counts(dropna=False).to_frame("count")
    t["share"] = (t["count"] / t["count"].sum()).round(4)
    freq_tables[c] = t

freq_tables["channel"]

,count,share
channel,,
Facebook Ads,1461,0.2922
Emailing,1428,0.2856
Instagram Ads,1189,0.2378
LinkedIn,922,0.1844


In [5]:
for c, t in freq_tables.items():
    t.to_csv(f"02_results/univarie_freq_{c}.csv")

In [6]:
ct_channel_status = pd.crosstab(df["channel"], df["status"], normalize="index").round(4)
ct_channel_status_count = pd.crosstab(df["channel"], df["status"])

ct_channel_status, ct_channel_status_count

(status         Client     MQL     SQL
 channel                              
 Emailing       0.1282  0.5742  0.2976
 Facebook Ads   0.1170  0.5763  0.3066
 Instagram Ads  0.1093  0.5786  0.3120
 LinkedIn       0.1399  0.4924  0.3677,
 status         Client  MQL  SQL
 channel                        
 Emailing          183  820  425
 Facebook Ads      171  842  448
 Instagram Ads     130  688  371
 LinkedIn          129  454  339)

In [7]:
ct_channel_status.to_csv("02_results/bivarie_channel_x_status_share.csv")
ct_channel_status_count.to_csv("02_results/bivarie_channel_x_status_count.csv")

In [8]:
ct_region_status = pd.crosstab(df["region"], df["status"], normalize="index").round(4)
ct_region_status_count = pd.crosstab(df["region"], df["status"])

ct_region_status.to_csv("02_results/bivarie_region_x_status_share.csv")
ct_region_status_count.to_csv("02_results/bivarie_region_x_status_count.csv")

In [9]:
ct_device_status = pd.crosstab(df["device"], df["status"], normalize="index").round(4)
ct_device_status_count = pd.crosstab(df["device"], df["status"])

ct_device_status.to_csv("02_results/bivarie_device_x_status_share.csv")
ct_device_status_count.to_csv("02_results/bivarie_device_x_status_count.csv")

In [10]:
ct_size_status = pd.crosstab(df["company_size"], df["status"], normalize="index").round(4)
ct_size_status_count = pd.crosstab(df["company_size"], df["status"])

ct_size_status.to_csv("02_results/bivarie_company_size_x_status_share.csv")
ct_size_status_count.to_csv("02_results/bivarie_company_size_x_status_count.csv")

In [11]:
camp_channel = camp.groupby("channel").agg(
    cost=("cost","sum"),
    impressions=("impressions","sum"),
    clicks=("clicks","sum"),
    conversions=("conversions","sum"),
    n_leads=("n_leads","sum")
).reset_index()

camp_channel["ctr"] = camp_channel["clicks"] / camp_channel["impressions"]
camp_channel["conv_rate"] = camp_channel["conversions"] / camp_channel["clicks"]
camp_channel["cpl"] = camp_channel["cost"] / camp_channel["n_leads"]
camp_channel["cpa"] = camp_channel["cost"] / camp_channel["conversions"]

camp_channel = camp_channel.round(4)
camp_channel

,channel,cost,impressions,clicks,conversions,n_leads,ctr,conv_rate,cpl,cpa
0,Emailing,7679,242212,5951,431,4284,0.0246,0.0724,1.7925,17.8167
1,Facebook Ads,9370,264536,5450,354,4383,0.0206,0.0650,2.1378,26.4689
2,Instagram Ads,7764,327821,10208,698,3567,0.0311,0.0684,2.1766,11.1232
3,LinkedIn,21928,332431,6747,486,2766,0.0203,0.0720,7.9277,45.1193


In [12]:
camp_channel.to_csv("02_results/bivarie_kpi_by_channel.csv", index=False)

In [13]:
df.shape, camp.shape
df["status"].value_counts()
df["channel"].value_counts()

channel
Facebook Ads     1461
Emailing         1428
Instagram Ads    1189
LinkedIn          922
Name: count, dtype: int64